# 20160923 IMSE 

Try to use Jupyter/Python Kernel/ to manage my worktime/sparetime

尝试使用Jupyter来管理我的加班/请假信息，管理我的历史，给出今年，近月 加班时间/请假时间

定好需求/设计格式/通用/设计函数

> 20160921 3.5hours STA_JOB设计

> 20160923 2hours SDM_JOB设计


In [5]:
import datetime

run_timestamp = datetime.datetime.now()
print( 'This year is ' + str(run_timestamp.year ) )
print( 'This month is ' + str(run_timestamp.month ) )

This year is 2016
This month is 9
